# Needleman Wunsch Algorithm
## Task
Compute an optimal global alignment for the two sequences **TATAAT** and **TTACGTAAGC** using the following column scoring function:

$
\varphi(a,b)=
  \begin{cases}
    -4  & \quad \text{if } a=- \text{ or } b=-\\
    3   & \quad \text{if } a=b\\
    -2  & \quad \text{if } a\neq b
  \end{cases}
$

You can do this using pen & paper **or** by implementing the Needleman-Wunsch algorithm and using your implementation (hand in the source code as well as the alignment!).
## Input Sequences

In [80]:
s = "TATAAT"
t = "TTACGTAAGC"

## Scoring Function
$
\varphi(a,b)=
  \begin{cases}
    -4  & \quad \text{if } a=- \text{ or } b=-\\
    3   & \quad \text{if } a=b\\
    -2  & \quad \text{if } a\neq b
  \end{cases}
$

In [81]:
def score(a, b):
    if a == "-" or b == "-":
        return -4               # Gap score
    elif a == b:
        return 3                # Match
    else:
        return -2               # Mismatch


## Create Scoring Table

The first part is to construct and populate the scoring table for the DP.

* **i** is our **row position**
* **j** is our **column position**
* a cell may be retrived with **score_table[j][i]**
* a cell with a relative position of **i-1** and **j-1** will be called the **top left neighbour**.
* a cell with a relative position of **i-1** and **j** will be called the **left neighbour**.
* a cell with a relative position of **i** and **j-1** will be called the **top neighbour**.

Filling the scoring table could be reduced in a single nested loop to gain a small performance improvment.

We start by initializing the complete table with 0.

Initializing just the first row and columns with 0 and the rest with None would be more save. Missing calculations in further steps would become more obviouse.

In [90]:
score_table = [[0 for i in range(len(s)+1)] for j in range(len(t)+1)]

First column is populated with the gab score, provided by our score function.
Note, we skip the first entry as it is to be initialized with 0!

In [91]:
for j in range(len(score_table))[1:]:
    score_table[j][0] = j*score(t[j-1], "-")

First row is populated with the gab score, provided by our score function.
Note, we skip the first entry as it is to be initialized with 0!

In [92]:
for i in range(len(score_table[0]))[1:]:
    score_table[0][i] = i*score(s[i-1], "-")

Now the table is populated with the respective scores left to right and top down.
Note, we skip the first row and column as they are already finished!

In [93]:
for j in range(len(score_table))[1:]:                       # Iterate columns starting with the second.
    for i in range(len(score_table[j]))[1:]:                # Iterate row cells for this column, starting with the second.
        score_table[j][i] = max([                           # Save the maximal score for the current cell.
                                                            # Note: We have to shift the position in the string one to the left as we added a prefix for our table.
            score_table[j-1][i-1] + score(t[j-1], s[i-1]),  # Score of the top left neighbour + calculated score for current alignment.
            score_table[j][i-1] + score("-", s[i-1]),        # Score of left neighbour + gap score.
            score_table[j-1][i] + score(t[j-1], "-")        # Score of top neighbour + gap score.
            ])


## Backtracking Scoring Table

The second part is backtracking an optimal solution from the scoring table.

There is multiple options to find an optimal solution:
1. Save the information how a specific cell has been reached while constructing the score table.
2. Calculate the for each 

We only backtrack one arbitrary possible (and optimal) path.

Set the last cell as starting point.

In [101]:
j = len(score_table)-1      # Last column position.
i = len(score_table[j])-1   # Last row position.
alignment = ([],[])         # Here will store our backltracking result.
pos = [(i,j)]               # Backtracked path.

In [102]:
while(i > 0 or j > 0):

    # print("Current j:{} i:{} score:{}".format(j,i,score_table[j][i]))
    # print("Top j:{} i:{} score:{}".format(j,i-1,score_table[j][i-1]))
    # print("Left j:{} i:{} score:{}".format(j-1,i,score_table[j-1][i]))

    # Score if the previouse cell would have been the top neighbour.
    # Gap case
    top = score_table[j-1][i] + score(t[j-1], "-")

    # Score if the previouse cell would have been the left neighbour.
    # Gap case
    left = score_table[j][i-1] + score("-", s[i-1])

    # Score if the previouse cell would have been the top-left neighbour.
    # Match/Mismatch case
    top_left = score_table[j-1][i-1] + score(t[j-1], s[i-1])

    # print("Top:{}, Left:{}, Top-Left:{}".format(top, left, top_left))

    if score_table[j][i] == top_left:
        i = i-1
        j = j-1
        alignment[0].insert(0, s[i])
        alignment[1].insert(0, t[j])
    elif score_table[j][i] == top:
        j = j-1
        alignment[0].insert(0, "-")
        alignment[1].insert(0, t[j])
    elif score_table[j][i] == left:
        i = i-1
        alignment[0].insert(0, s[i])
        alignment[1].insert(0, "-")
    else:
        raise ValueError("Scores do not match! Fatal error while Backtracking!")
    
    pos.append((i,j))

print(s)
print(alignment[0])
print(alignment[1])
print(t)

TATAAT
['-', 'T', 'A', '-', '-', 'T', 'A', 'A', '-', 'T']
['T', 'T', 'A', 'C', 'G', 'T', 'A', 'A', 'G', 'C']
TTACGTAAGC


## Now let's make it functional!

In [107]:
def alignment_fn(s: str, t: str, score_fn) -> tuple:
    # Table to safe the calculated alignment scores.
    score_table = [[0 for i in range(len(s)+1)] for j in range(len(t)+1)]

    # Initialize first row with gap scores.
    for i in range(len(score_table[0]))[1:]:
        score_table[0][i] = i*score(s[i-1], "-")

    # Initialize first column with gap scores.
    for j in range(len(score_table))[1:]:
        score_table[j][0] = j*score(t[j-1], "-")

    # Calculate remaining scores. Left to right and top down.
    for j in range(len(score_table))[1:]:                       # Iterate columns starting with the second.
        for i in range(len(score_table[j]))[1:]:                # Iterate row cells for this column, starting with the second.
            score_table[j][i] = max([                           # Save the maximal score for the current cell.
                                                                # Note: We have to shift the position in the string one to the left as we added a prefix for our table.
                score_table[j-1][i-1] + score(t[j-1], s[i-1]),  # Score of the top left neighbour + calculated score for current alignment.
                score_table[j][i-1] + score("-", s[i-1]),        # Score of left neighbour + gap score.
                score_table[j-1][i] + score(t[j-1], "-")        # Score of top neighbour + gap score.
                ])

    # Set initial parameters for backtracking.
    j = len(score_table)-1      # Last column position.
    i = len(score_table[j])-1   # Last row position.
    alignment = ([],[])         # Here will store our backltracking result.
    path = [(i,j)]               # Backtracked path.

    # Backtracking
    while(i > 0 or j > 0):
        # Score if the previouse cell would have been the top neighbour.
        # Gap case
        top = score_table[j-1][i] + score(t[j-1], "-")

        # Score if the previouse cell would have been the left neighbour.
        # Gap case
        left = score_table[j][i-1] + score("-", s[i-1])

        # Score if the previouse cell would have been the top-left neighbour.
        # Match/Mismatch case
        top_left = score_table[j-1][i-1] + score(t[j-1], s[i-1])

        if score_table[j][i] == top_left:
            i = i-1
            j = j-1
            alignment[0].insert(0, s[i])    # Prepend the characters to the list, as we are moving in reverse.
            alignment[1].insert(0, t[j])
        elif score_table[j][i] == top:
            j = j-1
            alignment[0].insert(0, "-")
            alignment[1].insert(0, t[j])
        elif score_table[j][i] == left:
            i = i-1
            alignment[0].insert(0, s[i])
            alignment[1].insert(0, "-")
        else:
            raise ValueError("Scores do not match! Fatal error while Backtracking!")
        
        path.append((i,j))   # Store the path taken while backtracking. While not required, this might come in handy for debugging and visualisation.

    return (alignment, score_table[len(t)][len(s)], path, score_table)

In [108]:
alignment_fn(s,t,score)

((['-', 'T', 'A', '-', '-', 'T', 'A', 'A', '-', 'T'],
  ['T', 'T', 'A', 'C', 'G', 'T', 'A', 'A', 'G', 'C']),
 -3,
 [(6, 10),
  (5, 9),
  (5, 8),
  (4, 7),
  (3, 6),
  (2, 5),
  (2, 4),
  (2, 3),
  (1, 2),
  (0, 1),
  (0, 0)],
 [[0, -4, -8, -12, -16, -20, -24],
  [-4, 3, -1, -5, -9, -13, -17],
  [-8, -1, 1, 2, -2, -6, -10],
  [-12, -5, 2, -1, 5, 1, -3],
  [-16, -9, -2, 0, 1, 3, -1],
  [-20, -13, -6, -4, -2, -1, 1],
  [-24, -17, -10, -3, -6, -4, 2],
  [-28, -21, -14, -7, 0, -3, -2],
  [-32, -25, -18, -11, -4, 3, -1],
  [-36, -29, -22, -15, -8, -1, 1],
  [-40, -33, -26, -19, -12, -5, -3]])

In [2]:
import graphviz

dot = graphviz.Digraph(comment='test')
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')
dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot.render('test-output/round-table.gv', view=True)

'test-output\\round-table.gv.pdf'